<a href="https://colab.research.google.com/github/okadatsuneo/GBFS_Import/blob/main/gbfstocsv_hc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#モジュールのインポート
import os
import pandas as pd
import json
from pathlib import Path

In [2]:
# Install XlsxWriter library
!pip install XlsxWriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 1.8 MB/s eta 0:00:00


In [3]:
#ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Googleドライブにあるstation_information.jsonを開く
path = Path('/content/drive/MyDrive/python/')
file_name_station_info = 'h_station_information.json'
file_path = path /  file_name_station_info
with open(file_path) as f:
    json_dict = json.load(f)

#'stations'の値を新たなリストlist_stationとする。
list_station = json_dict['data']['stations']

#list_stationの内容をDataFrameにする。
df = pd.DataFrame(list_station)

#ここからはHelloCyclingのファイルに必要な処理
#list_stationの中の'vehicle_type_capacity'の値をリストにする。
l_capacity = [ d['vehicle_type_capacity'] for d in list_station]
#さらにその中の'num_bikes_limit'の値をリストにする。
l_limitcap = [ d['num_bikes_limit'] for d in l_capacity]

#l_limitcapのリストをdfに変換する
df_cap =pd.DataFrame(l_limitcap, columns=['capacity'])

#station_idのcolumn名を変更する
df.rename(columns={'station_id':'gbfs:station_id'}, inplace=True)

#dfにdf_capを列として追加する。:df2
df2 = df.join(df_cap)

#インポートしやすいように'address'から都道府県名を取り出し、'addr:province'として列を作成する。
for index, row in df2.iterrows():
  #検索行の'address'の値を変数に入れる:address_check
  address_check = df2.at[index,'address']
  #3番目の文字が"都"の場合
  if address_check[2:3] == "都":
    #3番めまでの文字を'addr:province'に格納する。
    df2.at[index,'addr:province'] = address_check[:3]
  #3番目の文字が"道"の場合
  elif address_check[2:3] == "道":
    #3番めまでの文字を'addr:province'に格納する。
    df2.at[index,'addr:province'] = address_check[:3]
  #3番目の文字が"府"の場合
  elif address_check[2:3] == "府":
    #3番めまでの文字を'addr:province'に格納する。
    df2.at[index,'addr:province'] = address_check[:3]
  #3番目の文字が"県"の場合
  elif address_check[2:3] == "県":
    #3番めまでの文字を'addr:province'に格納する。
    df2.at[index,'addr:province'] = address_check[:3]
  #4番目の文字が"県"の場合
  elif address_check[3:4] == "県":
    #4番めまでの文字を'addr:province'に格納する。
    df2.at[index,'addr:province'] = address_check[:4]

#出力に必要な列だけ選択する:df3
df3 = df2[['lat','lon', 'name', 'gbfs:station_id', 'capacity', 'addr:province']].copy()

#df3にOSM用の固定値を追加
df3['amenity'] = 'bicycle_rental'
df3['brand'] = 'HELLO CYCLING'
df3['brand:ja'] = 'ハローサイクリング'
df3['brand:wikidata'] = 'Q91231927'
df3['network'] = 'HELLO CYCLING'
df3['network:ja'] = 'ハローサイクリング'
df3['network:wikidata'] = 'Q91231927'
df3['contact:website'] = 'https://www.hellocycling.jp'
df3['authentication:app'] = 'yes'
df3['fee'] = 'yes'
df3['gbfs:system_id'] = 'hellocycling'

#ファイル出力
df3.to_csv(path / 'h_stations.csv')
#確認用のexcelファイル
df3.to_excel(path / 'h_stations.xlsx', engine='xlsxwriter')
#データ数出力
#print(len(df2))
print("処理が完了しました。")

処理が完了しました。


備考
HelloCyclingのデータでstation_informationのすべてのデータを出力したら、
「IllegalCharacterError」のエラーが出た。
その回避のために「df.to_excel("test.xlsx", engine='xlsxwriter')」のようにengineでxlsxwriterを指定するとでなくなる。
なお、xlsxwriterはデフォルトではGoogle Colaboratoryに入ってないので、pipでインストールが必要。
pip install xlsxwriter
